### Imports

In [1]:
import os
import flywheel
import numpy as np
import pandas as pd
import nibabel as nib
from glob import glob
from os.path import join
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Custom imports

In [2]:
### Custom helpers
import sys
sys.path.append("classes")

# from Model import Model
from Cloud import Cloud
from Button import Button
from DicomDir import DicomDir 
from helpers import *
from Subject import Subject 

# ### Load objects
cloud = Cloud()

### Build model

In [3]:
def build_model(num_features):
    model = keras.Sequential([
        layers.Dense(64, activation=tf.nn.relu, input_shape=(num_features,)),
        layers.Dense(64, activation=tf.nn.relu),
        layers.Dense(32, activation=tf.nn.relu),
        layers.Dense(32, activation=tf.nn.relu),
        layers.Dense(1)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

    model.compile(
        loss='mean_squared_error',
        optimizer=optimizer,
        metrics=['mean_absolute_error', 'mean_squared_error'],
    )
    return model

NameError: name 'df' is not defined

### Run

In [ ]:
EPOCHS = 20
histories = list()

model = None
projects = ["engage"]
tasks = ["gonogo", "conscious", "nonconscious"]
time_sessions = ["000_data_archive"]

### Iterate over projects
for project in projects:
#         time_sessions = project_config["time_sessions"]
#         subjects = load_project_subjects(project)
    subjects = pd.read_csv("/Volumes/group/PANLab_Datasets/ENGAGE/ENGAGE_BV_sublist_n108_Final_List.csv")
    subjects = subjects.subNum

    ### Iterate over time_session/subject
    for time_session in time_sessions:
        for subject_id in subjects:
            print("Training on subject %s" % subject_id)
            ### Set logger info
#                 logger.set(project=project, subject=subject, time_session=time_session, task=task)

            ### Load subject specific data
            subject = Subject(subject_id)

            ### Load the button load files
            buttons = Button()
            buttons.find_subject_session_logs(project, subject_id, time_session)
            
            ### Skip subject if they have no onsets
            if len(buttons.onsets) == 0:
                print("No button logs for %s %s" % (subject_id, time_session))
                continue
                
            for i, task in enumerate(tasks):
                print("Training on task %s" % task)
                
                ###
                onsets = buttons.onsets[i]
                
                ### Download task dicom from Flywheel
                dicom_path = cloud.download(project, subject_id, time_session, task)

                ### Load dicoms and cut the first 3 volumes
                fmri_data = DicomDir(dicom_path)
                fmri_data.cut_volumes(3)

                for num_volume in range(1, fmri_data.num_volumes - 2):
                    print("Training on volume %d" % num_volume)
                    
                    ### Generate the training data from all the features
                    df = gen_data(project, task, onsets, fmri_data, subject, num_volume)
                    
                    ### Convert float64 values to float32 to save memory
                    df = convert_float64_to_float32(df)
                    
                    ### Get target values
                    labels = df.pop("signal")
                    
                    ### Initialize model
                    if model is None:
                        model = build_model(df.shape[1])
                        
                    ### Fit the model
                    record = model.fit(x=df, y=labels, validation_split=0.2, verbose=False, epochs=EPOCHS)
                    
                    ### Save the model weights as a checkpoint
                    model.save_weights('my_model_weights.h5')
                    
                    ### Save the history
                    histories.append(pd.DataFrame(record.history))

Training on subject PA30563
Training on task gonogo
unzipping /Users/pbezuhov/tmp/engage-PA30563-000_data_archive-gonogo.dicom.zip...


  0%|          | 7/6930 [00:00<01:46, 64.89it/s]

removing old path /Users/pbezuhov/tmp/engage-PA30563-000_data_archive-gonogo.dicom.zip...
loading dicoms from /Users/pbezuhov/tmp/engage-PA30563-000_data_archive-gonogo...
Sorting dicoms by trigger time...


100%|██████████| 6930/6930 [02:01<00:00, 57.09it/s]


Training on volume 1


...

100%|██████████| 1050/1050 [00:24<00:00, 42.44it/s] 
W0902 20:28:48.085379 4619183552 training.py:510] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Training on volume 5


100%|██████████| 1050/1050 [00:20<00:00, 51.94it/s] 
W0902 20:34:32.747190 4619183552 training.py:510] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Training on volume 6


  1%|          | 175/14336 [00:03<04:20, 54.26it/s]

### Remember to save the history before overwriting!

In [ ]:
history = pd.concat(histories)
history.to_csv("history1.csv") ### FIXME: put datetime in name